In [1]:
import tensorflow as tf
import numpy as np
from ptr_decoder import pointer_decoder
%load_ext autoreload
%autoreload 2

In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
with tf.device("/gpu:1"):
    encoder_inputs = [tf.constant(np.array([[0.5, 0.5]]), dtype=tf.float32)] * 4
    cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=10, state_is_tuple=True)
    encoder_output, final_state = tf.nn.rnn(cell=cell, dtype=tf.float32, inputs=encoder_inputs)
    attention_states = [tf.reshape(tensor=e, shape=(-1, 10)) for e in encoder_output]
    attention_states = tf.pack(axis=1, values=attention_states)
    decoder_inputs = [tf.constant(value=np.array([[0.5, 0.5]]), dtype=tf.float32)] * 4
    output = pointer_decoder(cell, decoder_inputs, final_state, attention_states)

In [4]:
sess.run(tf.initialize_all_variables())

In [14]:
sess.run(output[0])

array([[ 0.24572249,  0.26636925,  0.25572783,  0.23218039]], dtype=float32)

In [10]:
with tf.device("/gpu:1"):
    tf.get_variable_scope().reuse_variables()
    test = pointer_decoder(cell, decoder_inputs, attention_states=attention_states, 
                           encoder_inputs=encoder_inputs, feed_prev=True, initial_state=final_state)
    

In [13]:
sess.run(test[0])

array([[ 0.24572249,  0.26636925,  0.25572783,  0.23218039]], dtype=float32)